<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguage Natural

## Taller #10: Twitter
### Ilvar Dario Sanabria. Código: 616201058
`Fecha de entrega: Noviembre 14, 2020 (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

(Si exponen sus claves, les haré maldades 😈)

❌🎃❌ ¡No Halloween!

In [100]:
# Instalamos las librearias
import pandas as pd
import tweepy

In [101]:
# Leer las Claves para acceder al api de twiter

keys = pd.read_csv("c:/tmp/KeysAPI_twiter.txt", header=None)
keys = dict(zip(keys[0],keys[1]))

In [102]:
# Solicitar el acceso a través API

auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True)

In [103]:
# Leemos las coord de ciudades desde un archivo

ciudades = pd.read_csv("c:/tmp/co_small.csv")
ciudades.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Bogotá,4.649178,-74.062827,Colombia,CO,Bogotá,primary,7772000.0,6333661.0
1,Medellín,6.251840,-75.563591,Colombia,CO,Antioquia,admin,3297000.0,1999979.0
2,Cali,3.437222,-76.522500,Colombia,CO,Valle del Cauca,admin,2254000.0,2178836.0
3,Barranquilla,10.963889,-74.796389,Colombia,CO,Atlántico,admin,1798000.0,1244491.0
4,Bucaramanga,7.125393,-73.119804,Colombia,CO,Santander,admin,1009000.0,571820.0


In [104]:
# Recolectamos los datos relacionados con la palabra #selección# (por el partido de futbol !)
all_tweets = []

for ciudad in ciudades.iterrows() :
    c = ciudad[1]
    gc = f"{c.lat},{c.lng},70km"
    c_name = c.city
      
    tweets = tweepy.Cursor(api.search,
                        q=['selección -RT'],
                        lang='es',
                        geocode=gc,
                        tweet_mode='extended',
                        include_rts=False).items(20)


    t = []
 
    for tweet in tweets:
        temp = []
        temp.append(tweet.full_text)
        temp.append(tweet.created_at)
        temp.append(c_name)
        temp.append(tweet.user.location)
        temp.append(tweet.id)
        temp.append(f"https://twitter.com/i/web/status/{tweet.id}")
 
        t.append(temp)
    
    all_tweets.extend(t)
    print(c_name)
    

Bogotá
Medellín
Cali
Barranquilla
Bucaramanga
Cartagena
Cúcuta
Pereira
Santa Marta
Ibagué
Pasto
Manizales
Villavicencio
Neiva
Armenia
Valledupar
Montería
Sincelejo
Popayán
Tunja
Ríohacha
Florencia
Quibdó
Arauca
Yopal
Leticia
San Andrés
San José del Guaviare
Mocoa
Puerto Carreño
Mitú
Inírida


In [105]:
#Creamos un dataFrame con los datos recolectados
datos_tw = pd.DataFrame(all_tweets)
datos_tw.columns = ['tweet', 'date', 'city', 'user_location', 'id', 'link']

datos_tw.head()

,tweet,date,city,user_location,id,link
0,@CaracolTV que paso en el noticiero...olvidaro...,2020-11-14 01:55:58,Bogotá,bogota,1327430039017820161,https://twitter.com/i/web/status/1327430039017...
1,@VickyDavilaH Toda la selección jugó a nada. U...,2020-11-14 01:52:33,Bogotá,Bogotá,1327429181525909504,https://twitter.com/i/web/status/1327429181525...
2,Duvan zapata es muy malo en la selección,2020-11-14 01:51:01,Bogotá,Bogota,1327428794605596672,https://twitter.com/i/web/status/1327428794605...
3,@yeziddaniel @FCFSeleccionCol @mluciaramirez E...,2020-11-14 01:51:00,Bogotá,Bogotá,1327428788670648321,https://twitter.com/i/web/status/1327428788670...
4,@ch107 @AT_P8 fácil porque en la Juve cuando l...,2020-11-14 01:50:56,Bogotá,Colombia,1327428770626736129,https://twitter.com/i/web/status/1327428770626...


In [106]:
#Se define la funcion de limpieza de datos
import re
 
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'
 
def text_clean(row):    
    text = row['tweet']
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]

    for link in links:
        text = text.replace(link,"")

    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]

    for hashtag in hashtags:
         text = text.replace(hashtag,"")

    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]

    for mention in mentions:
         text = text.replace(mention,"")

    return text, links, hashtags, mentions

In [107]:
datos_tw[['clean', 'links', 'hashtags', 'mentions']] = datos_tw.apply(text_clean, axis=1, result_type='expand')

datos_tw.head()

,tweet,date,city,user_location,id,link,clean,links,hashtags,mentions
0,@CaracolTV que paso en el noticiero...olvidaro...,2020-11-14 01:55:58,Bogotá,bogota,1327430039017820161,https://twitter.com/i/web/status/1327430039017...,que paso en el noticiero...olvidaron las noti...,[],[],[@CaracolTV]
1,@VickyDavilaH Toda la selección jugó a nada. U...,2020-11-14 01:52:33,Bogotá,Bogotá,1327429181525909504,https://twitter.com/i/web/status/1327429181525...,Toda la selección jugó a nada. Un equipo perd...,[],[],[@VickyDavilaH]
2,Duvan zapata es muy malo en la selección,2020-11-14 01:51:01,Bogotá,Bogota,1327428794605596672,https://twitter.com/i/web/status/1327428794605...,Duvan zapata es muy malo en la selección,[],[],[]
3,@yeziddaniel @FCFSeleccionCol @mluciaramirez E...,2020-11-14 01:51:00,Bogotá,Bogotá,1327428788670648321,https://twitter.com/i/web/status/1327428788670...,El trino obviamente! La selección se recupe...,[],[],"[@yeziddaniel, @FCFSeleccionCol, @mluciaramirez]"
4,@ch107 @AT_P8 fácil porque en la Juve cuando l...,2020-11-14 01:50:56,Bogotá,Colombia,1327428770626736129,https://twitter.com/i/web/status/1327428770626...,"fácil porque en la Juve cuando la embarra, t...",[],[],"[@ch107, @AT_P8]"


###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

❌🎃❌ ¡No Halloween!

### Mostramos los emojis que estan en los twits

In [108]:

import regex

import emoji
 
def get_emojis(text):
    emoji_list = []

    data = regex.findall(r'\X', text)

    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
             emoji_list.append(word)
 
    return emoji_list
 
datos_tw['emojis'] = datos_tw['clean'].apply(lambda text: get_emojis(text))

In [109]:
datos_tw['emojis']

0      []
1      []
2      []
3      []
4      []
       ..
517    []
518    []
519    []
520    []
521    []
Name: emojis, Length: 522, dtype: object

In [110]:
## Importamos las librerias necesarias para realizar una grafica
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
 
import matplotlib.pyplot as plt

%matplotlib inline

init_notebook_mode(connected=True)

### Imprimimos una serie de tiempo

In [114]:

datos_tw['fecha'] = datos_tw.date.dt.date

temp = pd.DataFrame(datos_tw.fecha.value_counts()).reset_index()

temp.columns = ['fecha', 'cnt']
temp = temp.sort_values('fecha')

temp

,fecha,cnt
8,2020-11-05,1
3,2020-11-06,2
6,2020-11-07,1
7,2020-11-08,1
9,2020-11-09,1
4,2020-11-10,1
5,2020-11-11,1
2,2020-11-12,2
1,2020-11-13,141
0,2020-11-14,371


In [115]:
trace = go.Scatter(x=temp.fecha.values,
                    y=temp.cnt.values,
                    text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.fecha.values,temp.cnt.values)],
                    hoverinfo='text',
                    mode='lines+markers',
                    name='Días',
                    line={'color':'#1DA1F2'})
 
layout = go.Layout(title='Número de tuits por día') 
fig = go.Figure(data=[trace], layout=layout)

iplot(fig)

### Diagrama de barras de la cantidad de emojis

In [116]:
from collections import Counter
 
temp_emojis = pd.DataFrame(list(zip(Counter(sum(datos_tw.emojis.values,[])).keys(),Counter(sum(datos_tw.emojis.values,[])).values())))

temp_emojis.columns = ['emoji', 'cnt']
temp_emojis.sort_values('cnt', ascending=False, inplace=True)

temp_emojis.head()

,emoji,cnt
1,🤣,31
6,🇨🇴,18
55,🤡,15
20,❤,9
21,🇺🇾,6


In [17]:
trace = go.Bar(x=temp_emojis.emoji.values[:15],
                y=temp_emojis.cnt.values[:15])
 
layout = go.Layout(title='Emojis más usados')
 
fig = go.Figure(data=[trace], layout=layout)

iplot(fig)

### Hacemos un Mapa de calor de las palabras 

In [117]:
#Se importan las librerias para quitar las stopwors y hacer TFID 
import re

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

stopwords_sp = stopwords.words('spanish')
 
def pre_procesado(texto):
     texto = texto.lower()
     texto = re.sub(r"[\W\d_]+", " ", texto)
     texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
     return texto
 
tfidf_vect = TfidfVectorizer(preprocessor=pre_procesado, ngram_range=(1,2), max_features=50)

tfidf = tfidf_vect.fit_transform(datos_tw.clean.values)

In [118]:
# Se obtienen las distancias entre palabras
from sklearn.metrics.pairwise import cosine_similarity
 
voc = [k for k,v in sorted(tfidf_vect.vocabulary_.items(), key=lambda kv: kv[1])]

temp = pd.DataFrame(tfidf.toarray())
temp = cosine_similarity(temp.T.values)
temp = pd.DataFrame(temp)
temp.columns = voc
temp.index = voc

temp

,ahora,así,barranquilla,bien,camiseta,colombia,colombiano,da,derrota,ecuador,...,selección,selección colombia,si,siempre,solo,tan,técnico,uruguay,venezuela,vergara
ahora,1.000000,0.052785,0.000000,0.000000,0.000000,0.028006,0.000000,0.000000,0.080870,0.114273,...,0.123846,0.000000,0.132759,0.000000,0.013368,0.070561,0.044563,0.000000,0.000000,0.126760
así,0.052785,1.000000,0.000000,0.000000,0.000000,0.142381,0.000000,0.000000,0.000000,0.034567,...,0.077549,0.038918,0.064233,0.053631,0.000000,0.000000,0.163552,0.092126,0.000000,0.034759
barranquilla,0.000000,0.000000,1.000000,0.040128,0.023634,0.219473,0.050884,0.021697,0.044305,0.000000,...,0.146012,0.159303,0.039393,0.000000,0.000000,0.000000,0.000000,0.304599,0.053771,0.000000
bien,0.000000,0.000000,0.040128,1.000000,0.000000,0.063732,0.000000,0.042011,0.000000,0.000000,...,0.058189,0.083241,0.089710,0.000000,0.041715,0.000000,0.032834,0.058691,0.000000,0.000000
camiseta,0.000000,0.000000,0.023634,0.000000,1.000000,0.011150,0.000000,0.000000,0.000000,0.000000,...,0.069009,0.014564,0.265286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
colombia,0.028006,0.142381,0.219473,0.063732,0.011150,1.000000,0.033490,0.014281,0.109985,0.167173,...,0.196857,0.798144,0.092131,0.051857,0.091549,0.083456,0.089157,0.287426,0.000000,0.008253
colombiano,0.000000,0.000000,0.050884,0.000000,0.000000,0.033490,1.000000,0.298123,0.000000,0.000000,...,0.034972,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
da,0.000000,0.000000,0.021697,0.042011,0.000000,0.014281,0.298123,1.000000,0.000000,0.032834,...,0.067130,0.018652,0.000000,0.000000,0.000000,0.163992,0.087438,0.000000,0.000000,0.000000
derrota,0.080870,0.000000,0.044305,0.000000,0.000000,0.109985,0.000000,0.000000,1.000000,0.120700,...,0.060119,0.055273,0.000000,0.000000,0.000000,0.000000,0.000000,0.066871,0.000000,0.000000
ecuador,0.114273,0.034567,0.000000,0.000000,0.000000,0.167173,0.000000,0.032834,0.120700,1.000000,...,0.061072,0.135127,0.039284,0.076727,0.086001,0.047611,0.000000,0.039075,0.116476,0.000000


#### Se hace la grafica de calor con base en las distancias

In [119]:
import scipy.cluster.hierarchy as sch

import numpy as np
 
pairwise_distances = sch.distance.pdist(temp)
linkage = sch.linkage(pairwise_distances, method='ward')

idx_to_cluster_array = sch.fcluster(linkage, pairwise_distances.max() * 1, criterion='distance')
idx = np.argsort(idx_to_cluster_array)

temp = temp.copy()

temp2 = temp.iloc[idx, :].T.iloc[idx, :]

my_idx = idx_to_cluster_array

In [22]:
trace = go.Heatmap(z=temp2.values.tolist(),
                    x=temp2.index.values,
                    y=temp2.columns.values,
                    colorscale='Blues')
 
layout = go.Layout(title=' Mapa de calor entre palabras - Twits relacionados con la "Selección"', width=800, height=800)
 
fig = go.Figure(data=[trace],layout=layout)

iplot(fig)